# 3-1. Video Processing Basic

In [ ]:
!pip3 install --upgrade mxnet-cu101 > /dev/null
!pip3 install --upgrade gluoncv > /dev/null

## video download

### direct download

In [ ]:
from gluoncv import utils
video_path = 'https://raw.githubusercontent.com/dmlc/web-data/master/gluoncv/tracking/Coke.mp4'
coke = utils.download(video_path)

youtube download

In [ ]:
!pip3 install --upgrade youtube-dl

In [ ]:
import youtube_dl

ydl_opts = {  
    # 'format': 'mp4' 
    'format': 'best[height<=480][ext=mp4]' 
  } 
with youtube_dl.YoutubeDL(ydl_opts) as ydl: 
    info_dict = ydl.extract_info('https://www.youtube.com/watch?v=C-CefuZ6h1k&feature=emb_err_woyt', download=True)    
    filename = ydl.prepare_filename(info_dict)


In [ ]:
def get_youtube_video(youtube_url):
  import youtube_dl

  ydl_opts = {  
    # 'format': 'mp4' 
    'format': 'best[height<=480][ext=mp4]' 
    } 
  with youtube_dl.YoutubeDL(ydl_opts) as ydl: 
      info_dict = ydl.extract_info(youtube_url, download=True)    
      filename = ydl.prepare_filename(info_dict)
  return filename

In [ ]:
son = get_youtube_video('https://www.youtube.com/watch?v=C-CefuZ6h1k&feature=emb_err_woyt')
print(son)

## video read/write

In [ ]:
from gluoncv.utils.filesystem import try_import_cv2
cv2 = try_import_cv2()

### video read

In [ ]:
video_frames = [] 
cap = cv2.VideoCapture(coke)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
print('Frame width:', width)
print('Frame height:', height)
print('Frame count:', frame_count)
print('FPS:', fps)
 
while(True):
    ret, img = cap.read()
    if not ret:
        break
    frame_no = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
    frame_msec = int(cap.get(cv2.CAP_PROP_POS_MSEC))

    print('{}. msec:{}'.format(frame_no, frame_msec)) 

    video_frames.append(img)

In [ ]:
def read_video(filename):
  import os 
  from gluoncv.utils.filesystem import try_import_cv2
  cv2 = try_import_cv2()  

  video_frames = [] 
  cap = cv2.VideoCapture(filename) 

  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  fps = cap.get(cv2.CAP_PROP_FPS)
  print('Frame width:', width)
  print('Frame height:', height)
  print('Frame count:', frame_count)
  print('FPS:', fps) 
   
  while(True):
    ret, img = cap.read()
    if not ret:
      break
    frame_no = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
    frame_msec = int(cap.get(cv2.CAP_PROP_POS_MSEC)) 

    print('{}. msec:{}'.format(frame_no, frame_msec)) 

    video_frames.append(img)


  return video_frames, width, height, fps, frame_count

### image write

In [ ]:
import os
save_dir = './images'
if not os.path.exists(save_dir):
  os.makedirs(save_dir)

for ind, frame in enumerate(video_frames):
  cv2.imwrite(os.path.join(save_dir, '%04d.jpg'%(ind+1)), frame)

### video write
- codec
  - DIVX
  - XVID
  - FMP4
  - X264
  - MJPG 

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'FMP4')

In [ ]:
out = cv2.VideoWriter('coke_write.mp4', fourcc, fps, (width, height))

In [ ]:
for ind, frame in enumerate(video_frames):
  out.write(frame) 
out.release()

### fps 조정

#### fps/2 = 1/2 배속

In [ ]:
out = cv2.VideoWriter('coke_write_slow.mp4', fourcc, fps/2, (width, height))
for ind, frame in enumerate(video_frames):
  out.write(frame) 
out.release()

#### fps/4 = 1/4 배속

In [ ]:
out = cv2.VideoWriter('coke_write_slow_4.mp4', fourcc, fps/4, (width, height))
for ind, frame in enumerate(video_frames):
  out.write(frame) 
out.release()

#### fps*2 = 2배속

In [ ]:
out = cv2.VideoWriter('coke_write_fast2.mp4', fourcc, fps*2, (width, height))
for ind, frame in enumerate(video_frames):
  out.write(frame) 
out.release()

#### fps*4 = 4배속

In [ ]:
out = cv2.VideoWriter('coke_write_fast4.mp4', fourcc, fps*4, (width, height))
for ind, frame in enumerate(video_frames):
  out.write(frame) 
out.release()

### video size 조정
[참고](https://076923.github.io/posts/Python-opencv-8/)
- interpolation 속성
  - 확대
    - cv2.INTER_CUBIC	(바이큐빅 보간법)
      - 4*4 이웃 픽셀 참고, 퀄리티 좋음
    - cv2.INTER_LINEAR	(쌍 선형 보간법)
      - 2*2 이웃 픽셀 참고, 효율성 좋음
  - 축소
    - cv2.INTER_AREA	(영역 보간법)
      - 영역 정보를 이용하여 사용

#### 480p -> 720p

In [ ]:
ratio = 720/height
resize = (round(width*ratio), round(height*ratio))
out = cv2.VideoWriter('coke_write_720p.mp4', fourcc, fps, resize)
for ind, frame in enumerate(video_frames):
  frame = cv2.resize(frame, dsize=resize, interpolation=cv2.INTER_LINEAR)
  out.write(frame) 
out.release()

#### 480p -> 1080p

In [ ]:
ratio = 1080/height
resize = (round(width*ratio), round(height*ratio))
out = cv2.VideoWriter('coke_write_1080p.mp4', fourcc, fps, resize)
for ind, frame in enumerate(video_frames):
  frame = cv2.resize(frame, dsize=resize, interpolation=cv2.INTER_LINEAR)
  out.write(frame) 
out.release()

#### 480p -> 240p

In [ ]:
ratio = 240/height
resize = (round(width*ratio), round(height*ratio))
out = cv2.VideoWriter('coke_write_240p.mp4', fourcc, fps, resize)
for ind, frame in enumerate(video_frames):
  frame = cv2.resize(frame, dsize=resize, interpolation=cv2.INTER_AREA)
  out.write(frame) 
out.release()

### 영상 필터
- [참고](https://deep-learning-study.tistory.com/174?category=946336)

#### 스케치필터

In [ ]:
def pencil_sketch(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blr = cv2.GaussianBlur(gray, (0, 0), 3) 
    dst = cv2.divide(gray, blr, scale=255) 
    dst = cv2.cvtColor(dst, cv2.COLOR_GRAY2BGR)
    return dst

In [ ]:
out = cv2.VideoWriter('coke_write_sketch.mp4', fourcc, fps, (width, height))
for ind, frame in enumerate(video_frames):
  frame = pencil_sketch(frame)
  out.write(frame) 
out.release()

#### 카툰필터

In [ ]:
def cartoon_filter(img):
    h, w = img.shape[:2]
    img2 = cv2.resize(img, (w//2, h//2))

    blr = cv2.bilateralFilter(img2, -1, 20, 7)
    edge = 255 - cv2.Canny(img2, 80, 120)
    edge = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR)
    dst = cv2.bitwise_and(blr, edge) 
    dst = cv2.resize(dst, (w, h), interpolation=cv2.INTER_NEAREST)
                                                                  
    return dst

In [ ]:
out = cv2.VideoWriter('coke_write_cartoon.mp4', fourcc, fps, (width, height))
for ind, frame in enumerate(video_frames):
  frame = cartoon_filter(frame)
  out.write(frame) 
out.release()

In [ ]:
def cartoon_filter2(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray, 5)
    edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 9)

    color = cv2.bilateralFilter(img, 9, 250, 250)
    cartoon = cv2.bitwise_and(color, color, mask=edges)

    return cartoon


In [ ]:
out = cv2.VideoWriter('coke_write_cartoon2.mp4', fourcc, fps, (width, height))
for ind, frame in enumerate(video_frames):
  frame = cartoon_filter2(frame)
  out.write(frame) 
out.release()

# 실습

## 1. 손흥민 영상을 다운받아서 
 - 1/2 배속
 - 1/4 배속 영상을 작성한다.
 - 파일명 : 
   - son_slow2.mp4, 
   - son_slow4.mp4

In [ ]:
son = get_youtube_video('https://www.youtube.com/watch?v=C-CefuZ6h1k&feature=emb_err_woyt')
print(son)

## 2. 안개길 출근영상을 
- 스케치 필터, 
- 카툰 필터를 적용해서 영상을 만든다.
- 파일명:
  - fog_sketch.mp4
  - fog_cartoon.mp4

In [ ]:
fog_drive = get_youtube_video('https://www.youtube.com/watch?v=8YUJSu-C4Vc')
print(fog_drive)